In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import numpy as np
import tensorflow as tf
path = r'..\\embeddings\\'
id_dict = {}
word_dict = {}

matrix = []
dimensions = 500
with open(path + "m_base2021_wdiag_500d__100e_wc", 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        values = line.split()
        word = values[0].strip()
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values[1:], "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

260377


In [3]:
def find_candidate(sim,id_a,id_a_star,id_b):
    myList = [id_a, id_b, id_a_star];
    myList.sort();
    
    if(myList[0] != 0):
        index1 = np.argmax(sim[:myList[0]])
    else:
        index1 = None
    sim2 = sim[myList[0]+1:myList[1]]
    if len(sim2) > 0:
        index2 = np.argmax(sim2)
        index2 +=  myList[0]+1
    else:
        index2 = None
    sim3 = sim[myList[1]+1:myList[2]]
    if len(sim3) > 0:
        index3 = np.argmax(sim3)
        index3 +=  myList[1]+1
    else:
        index3 = None
    if(myList[2]+1 < len(sim)):
        index4 = np.argmax(sim[myList[2]+1:])
        index4 +=  myList[2]+1
    else:
        index4 = None
    myList = [index1, index2, index3,index4];
    values = []
    myList2 = []
    for index in myList:
        if index != None:
            values.append(sim[index])
            myList2.append(index)
    index = myList2[np.argmax(values)]
    return index

In [4]:
matrix_normalized = tf.nn.l2_normalize(matrix,axis = 1)

In [5]:
def find_nearest_k(searched_word,k):
    list = []
    id = id_dict[searched_word]
    searched_vector = matrix_normalized[id,:] 
    
    for word in id_dict:
        word_weights = matrix_normalized[id_dict[word]]
        loss = tf.tensordot(word_weights,searched_vector,axes = 1).numpy()
        list = insert(list,(word,loss))
        if len(list) > k:
            list = list[0:k+1]
    return list[0:k]

# Function to insert element
def insert(list, tuple):
    (word,n) = tuple
    if(len(list) == 0):
        list = [(word,n)]
    # Searching for the position
    for i in range(len(list)):
        (word_i,n_i) = list[i]
        if n_i < n:
            index = i
            break
      
    # Inserting n in the list
    list = list[:i] + [(word,n)] + list[i:]
    return list


print(find_nearest_k('batman',6)[1:])#this should produce one, for the same vectors
print(find_nearest_k('hogwarts',6)[1:])
print(find_nearest_k('turing',6)[1:])
print(find_nearest_k('hogwarts',6)[1:])
print(find_nearest_k('florida',6)[1:])
#print(find_nearest_k('object-oriented',6))
print(find_nearest_k('dancing',6)[1:])

[('superman', 0.5858298579170151), ('kozluk', 0.5275125769891488), ('joker', 0.49318277014450035), ('sason', 0.47949554598505956), ('robin', 0.4621860494247182)]
[('tsukuba', 0.46736731399622305), ('abukuma', 0.4584975510593807), ('hacettepe', 0.43486239675481353), ('gazala', 0.43321341864369983), ('ambohimanga', 0.42955873142800377)]
[('machine', 0.4746365447199307), ('alan', 0.4444189466775258), ('machines', 0.4433181988301288), ('deterministic', 0.43767846685098327), ('vending', 0.41870508490588076)]
[('tsukuba', 0.46736731399622305), ('abukuma', 0.4584975510593807), ('hacettepe', 0.43486239675481353), ('gazala', 0.43321341864369983), ('ambohimanga', 0.42955873142800377)]
[('miami', 0.6027603634498784), ('jacksonville', 0.5724042695584727), ('gainesville', 0.5618632119290442), ('gators', 0.5605428523360936), ('marlins', 0.5494419712596553)]
[('singing', 0.6069267893830713), ('dance', 0.5925366698287883), ('dancers', 0.5480954865196336), ('dancer', 0.5187918425483398), ('tap', 0.4947

## 3cosAdd 

In [6]:

def three_cos_predict_np(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix[id_a,:]
    weight_b = matrix[id_b,:]
    weight_a_star = matrix[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

# 3cosAdd normalised befor arithmetic

In [7]:
def three_cos_predict_np_norm(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

## 3cos Mult

In [8]:
def sim_01(matrix,weight):
    tmp = tf.tensordot(matrix,weight,axes=1)
    tmp = (tmp+1.0)/2.0
    return tmp

def predict_three_cos_mult(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        print(a in id_dict)
        print(a_star in id_dict)
        print(b in id_dict)
        print(b_star in id_dict)
        return None
    
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    
    nominator = sim_01(matrix_normalized,weight_a_star) * sim_01(matrix_normalized,weight_b) 
    denominator = sim_01(matrix_normalized,weight_a) + 0.0001
    sim = nominator / denominator
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

# Test all

In [9]:
count_sem_questions =  0
count_syn_questions =  0

count_sem_sucess      = [0,0,0]
count_syn_sucess      = [0,0,0]

errors = []


files = os.listdir('.\\datasets\\question-data')
for idx,file_name in enumerate(files):
    print(file_name)
    with open('.\\datasets\\question-data\\'+file_name,'r') as file:
        lines = file.readlines()
        local_sucesses    = [0,0,0]
        local_tasks_count = 0
        
        for line_id,line in enumerate(lines):
            if (line_id % 100 == 0):
                print(line_id)
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()
            #predicted_3cos_add      = three_cos_predict_np(a,a_star,b,b_star)
            predicted_3cos_add_norm = three_cos_predict_np_norm(a,a_star,b,b_star)
            predicted_3cos_mult     = predict_three_cos_mult(a,a_star,b,b_star)
            #print(a,a_star,b,b_star)
            #print(predicted_3cos_add,predicted_3cos_add_norm,predicted_3cos_mult)
            if(idx < 5):#first 5 are sem
                count_sem_questions += 1
                local_tasks_count   += 1
                
                #if predicted_3cos_add      == b_star:
                #    count_sem_sucess[0] += 1
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[1] += 1
                    count_sem_sucess[1] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[2] += 1
                    count_sem_sucess[2] += 1
            else:
                count_syn_questions += 1
                local_tasks_count   += 1
                
                #if predicted_3cos_add      == b_star:
                #    count_syn_sucess[0] += 1
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[1] += 1
                    count_syn_sucess[1] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[2] += 1
                    count_syn_sucess[2] += 1
        print(local_sucesses[0]/float(local_tasks_count))
        print(local_sucesses[1]/float(local_tasks_count))
        print(local_sucesses[2]/float(local_tasks_count))
        print('-------------------')

print('semantical')

print(count_sem_sucess[0]/float(count_sem_questions))
print(count_sem_sucess[1]/float(count_sem_questions))
print(count_sem_sucess[2]/float(count_sem_questions))
        
print('syntactical')
print(count_syn_sucess[0]/float(count_sem_questions))
print(count_syn_sucess[1]/float(count_sem_questions))
print(count_syn_sucess[2]/float(count_sem_questions))

print('overall')
print((count_syn_sucess[0]+count_sem_sucess[0])/float(count_sem_questions+count_sem_questions))
print((count_syn_sucess[1]+count_sem_sucess[1])/float(count_sem_questions+count_sem_questions))
print((count_syn_sucess[2]+count_sem_sucess[2])/float(count_sem_questions+count_sem_questions))

capital-common-countries.txt
0
100
200
300
400
500
0.0
0.9782608695652174
0.9683794466403162
-------------------
capital-world.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
0.0
0.8081343943412909
0.7694518125552609
-------------------
city-in-state.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
0.0
0.7904337251722741
0.783542764491285
-------------------
currency.txt
0
True
True
True
False
True
True
True
False
True
True
True
False
100
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
200
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
300
True
True
True
False
True
True
True
False
True
True
True
False
400
True
True
True
False
True
True
True
False
True
True
True
False
True
True

In [10]:
import winsound
winsound.Beep(440, 500)

In [11]:
(6780 + 6446) / (8869 + 10675)

0.6767294310274253

Spearman Rank

In [12]:
with open(r'.\datasets\wordsim353_sim_rel\wordsim_similarity_goldstandard.txt') as file:

    losses = []
    scores = []

    lines = file.readlines()
    sum_diff = 0
    for line in lines:
        if(len(line) <= 1):
            pass

        print("line" + line)
        word1, word2, score10 = line.split()
        score10 = float(score10)
        try:
            print(word1,word2)
            id1 = id_dict[word1]
            id2 = id_dict[word2]
            print(id1)
            print(id2)
            vector1 = matrix_normalized[id1,:]
            vector2 = matrix_normalized[id2,:]
            #print(vector1)
            #print(vector2)
            loss = sim_01(vector1,vector2)
        except: 
            loss = 0
        print(str(loss),str(score10/10.0))
        sum_diff += abs((score10/10.0) - loss)
        losses.append(loss)
        scores.append(score10/10.0)
    print('Average Distance between prediction and hand assigned is ' + str(sum_diff / len(lines)))

linetiger	cat	7.35

tiger cat
174824
1460
tf.Tensor(0.6855797830448571, shape=(), dtype=float64) 0.735
linetiger	tiger	10.00

tiger tiger
174824
174824
tf.Tensor(1.0, shape=(), dtype=float64) 1.0
lineplane	car	5.77

plane car
237142
260365
tf.Tensor(0.6674810252879313, shape=(), dtype=float64) 0.577
linetrain	car	6.31

train car
225926
260365
tf.Tensor(0.7481309099948014, shape=(), dtype=float64) 0.631
linetelevision	radio	6.77

television radio
173375
82259
tf.Tensor(0.8036571095473969, shape=(), dtype=float64) 0.6769999999999999
linemedia	radio	7.42

media radio
127720
82259
tf.Tensor(0.7205482150203729, shape=(), dtype=float64) 0.742
linebread	butter	6.19

bread butter
105268
70044
tf.Tensor(0.7535324001213031, shape=(), dtype=float64) 0.619
linecucumber	potato	5.92

cucumber potato
188967
37462
tf.Tensor(0.6231749710169194, shape=(), dtype=float64) 0.592
linedoctor	nurse	7.00

doctor nurse
114009
219162
tf.Tensor(0.6884326296812706, shape=(), dtype=float64) 0.7
lineprofessor	doctor

tf.Tensor(0.8017499039114144, shape=(), dtype=float64) 0.836
lineman	woman	8.30

man woman
88692
60261
tf.Tensor(0.84103095109875, shape=(), dtype=float64) 0.8300000000000001
lineman	governor	5.25

man governor
88692
165118
tf.Tensor(0.6233242357119303, shape=(), dtype=float64) 0.525
linemurder	manslaughter	8.53

murder manslaughter
151365
181602
tf.Tensor(0.7352174171408462, shape=(), dtype=float64) 0.853
lineopera	performance	6.88

opera performance
140937
161786
tf.Tensor(0.6973243086294987, shape=(), dtype=float64) 0.688
lineMexico	Brazil	7.44

Mexico Brazil
0 0.744
lineglass	metal	5.56

glass metal
220227
205240
tf.Tensor(0.7089070046187949, shape=(), dtype=float64) 0.5559999999999999
linealuminum	metal	7.83

aluminum metal
9809
205240
tf.Tensor(0.6920897618851979, shape=(), dtype=float64) 0.783
linerock	jazz	7.59

rock jazz
242138
65511
tf.Tensor(0.7457152309752371, shape=(), dtype=float64) 0.759
linemuseum	theater	7.19

museum theater
205119
32568
tf.Tensor(0.6740306570135383, s

tf.Tensor(0.6272389607083773, shape=(), dtype=float64) 0.265
lineopera	industry	2.63

opera industry
140937
255663
tf.Tensor(0.6303958957799976, shape=(), dtype=float64) 0.263
linevolunteer	motto	2.56

volunteer motto
9078
249214
tf.Tensor(0.6131959211700109, shape=(), dtype=float64) 0.256
linelisting	proximity	2.56

listing proximity
8419
245618
tf.Tensor(0.5424353844825853, shape=(), dtype=float64) 0.256
lineprecedent	collection	2.50

precedent collection
81750
176459
tf.Tensor(0.5732713993373044, shape=(), dtype=float64) 0.25
linecup	article	2.40

cup article
180411
167223
tf.Tensor(0.5931125657126379, shape=(), dtype=float64) 0.24
linesign	recess	2.38

sign recess
9724
157385
tf.Tensor(0.5508986101779542, shape=(), dtype=float64) 0.238
lineproblem	airport	2.38

problem airport
139423
250735
tf.Tensor(0.589936658113281, shape=(), dtype=float64) 0.238
linereason	hypertension	2.31

reason hypertension
247794
19634
tf.Tensor(0.5969185146309236, shape=(), dtype=float64) 0.231
linedirect

In [13]:
from scipy import stats
stats.spearmanr(losses, scores)

SpearmanrResult(correlation=0.5470115327416267, pvalue=3.074612596494663e-17)